In [1]:
from sympy import sqrt, I, series, limit, symbols, factorial, cos, sin, atan
from sympy.core.add import Add
from sympy.core.mul import Mul
from sympy.physics.secondquant import B, Bd, CreateBoson, AnnihilateBoson, Commutator, KroneckerDelta

In [2]:
# Eliminates all powers
def flatten(expr):
    expr = expr.expand()
    if expr.is_Add: return Add(*[flatten(arg) for arg in expr.args])
    elif expr.is_Mul: 
        factors = []
        for arg in expr.args:
            if arg.is_Pow and arg.exp.is_Integer and arg.exp > 0:
                factors.extend([arg.base] * int(arg.exp))
            else:
                factors.append(flatten(arg))
        return Mul(*factors, evaluate=False)
    elif expr.is_Pow and expr.exp.is_Integer and expr.exp > 0:
        return Mul(*[flatten(expr.base)] * int(expr.exp), evaluate=False)
    return expr

# Puts a polynomial expression of bosonic operators into the equivalent normal ordered form
def wick(expr):
    expr = flatten(expr)
    if expr.is_Add: return Add(*[wick(arg) for arg in expr.args])
    elif expr.is_Mul:
        commuting = [x for x in expr.args if not isinstance(x, AnnihilateBoson) and not isinstance(x, CreateBoson)]
        noncommuting = [x for x in expr.args if isinstance(x, AnnihilateBoson) or isinstance(x, CreateBoson)]
        n = len(noncommuting)
        if n == 0 or n == 1: return expr
        else:
            for i in range(n-1):
                c = Commutator(noncommuting[i], noncommuting[i+1])
                if isinstance(c, KroneckerDelta): c = 0
                if isinstance(noncommuting[i], AnnihilateBoson) and isinstance(noncommuting[i+1], CreateBoson):
                    expr1 = [c] + noncommuting[:i] + noncommuting[i+2:]
                    expr2 = noncommuting[:i] + [noncommuting[i+1], noncommuting[i]] + noncommuting[i+2:]
                    return wick(Mul(*commuting) * (wick(Mul(*expr1)) + wick(Mul(*expr2))))
            return expr
    else:
        return expr

# Computes the vacuum expectation value of a polynomial expression of bosonic operators
vev = lambda expr: Add(*[x for x in wick(expr).args if not any(isinstance(y, AnnihilateBoson) or isinstance(y, CreateBoson) for y in x.args)])

In [3]:
# Verifies the above functions work by using them to reproduce the main results of the paper:
# Quantum Theory of the Josephson Junction between Finite Islands

ng, x = symbols('n_g x', real = True)
N, EJ, EC = symbols("N E_J E_C", positive = True)
u_s, u_p, u_m, u_0 = symbols('u_s, u_p, u_m, u_0')
eps = sqrt(2*EC*EJ + EJ**2/N**2)
u_p = (EJ + N * eps) / sqrt(4 * N * eps * EJ)
u_m = (EJ - N * eps) / sqrt(4 * N * eps * EJ)
u_0 = ng * sqrt(2 * EC**2 * EJ / eps**3)
u_s = u_p + u_m

b = B('b')
bd = Bd('b')
a = u_p*b - u_m*bd + I*u_0*(u_p+u_m)
ad = u_p*bd - u_m*b - I*u_0*(u_p+u_m)
p = (a-ad)/(I*sqrt(2))
pp = p - ng/sqrt(N)

expr = -EC/2*(b*pp*ad*p*a*bd - pp*ad*p*a)
freq = eps + vev(expr)
assert((freq - series(freq, ng, 0, 3)).simplify()==0)
assert(freq.diff(ng).subs(ng,0)==0)
freq_0 = limit(freq.subs(ng, 0).subs(N, x * EJ/EC), x, "oo")
freq_2 = limit((N**2 * (freq.diff(ng, 2)/2).subs(ng, 0)).subs(N, x * EJ/EC), x, "oo")*(ng/N)**2
freq = (limit(((freq_0+freq_2)/sqrt(EC * EJ)).subs(EC, x * EJ), x, 0) * sqrt(EC * EJ)).simplify()
print('Transmon qubit frequency times Planck\'s constant:')
display(freq)

zeroth_order = vev(sqrt(N)*p)
first_order = vev(-ad*p*a/(4*sqrt(N)))
for k in range(1,5): first_order += EC*sqrt(N)/(2*eps*k*factorial(k)) * vev(p*bd**k) * (vev(b**k*pp*ad*p*a) + vev(b**k*ad*p*a*pp))
all_orders = (zeroth_order + first_order).simplify()
all_orders.diff(ng, 3).subs(ng, 0).simplify()
suscept = all_orders.diff(ng)
assert((suscept - series(suscept, ng, 0, 3)).simplify() == 0)
assert(suscept.diff(ng).subs(ng, 0) == 0)
suscept_0 = limit(suscept.subs(ng, 0).subs(N, x * EJ/EC), x, "oo")
suscept_2 = limit((N**4 * (suscept.diff(ng, 2)/2).subs(ng, 0)).subs(N, x * EJ/EC), x, "oo")*(ng**2)/(N**4)
suscept = suscept_0 + suscept_2
print('Transmon dimensionless charge susceptibility:')
display(suscept.simplify())

Transmon qubit frequency times Planck's constant:


sqrt(2)*sqrt(E_C)*sqrt(E_J)*(N**2 - n_g**2/4)/N**2

Transmon dimensionless charge susceptibility:


1 - 3*E_J*n_g**2/(4*E_C*N**4)

In [4]:
# define spin operators
S = symbols('S', positive = True)
th = symbols('theta', real = True)
a, ad = B('a'), Bd('a')
sz = S - ad*a
sp = sqrt(2*S)*(1-ad*a/(4*S))*a
sm = ad*sqrt(2*S)*(1-ad*a/(4*S))

sx = (sp+sm)/2
Sz = sz*cos(th)-sx*sin(th)
Sx = sx*cos(th)+sz*sin(th)

# define bosonic operators
hw = symbols(r'\hbar\omega', positive = True)
l = symbols(r'\lambda', real = True)
c, cd = B('c'), Bd('c')
b, bd = c - sqrt(2*S)*l*cos(th)/(4*hw), cd - sqrt(2*S)*l*cos(th)/(4*hw)

# define Hamiltonians
J, U = symbols(r'J U', positive = True)
gm = symbols(r'\gamma_-', real = True)
H_M = -J * Sx + U/(2*S)*Sz**2 - (gm/2)*Sz
H_L = hw*bd*b
H_LM = l/(2*sqrt(2*S))*Sz*(b+bd)
H = (H_M + H_L + H_LM).expand()

sqrtS = symbols(r'\sqrt{S}', positive = True)
H_ = []
max_order = 4
h = (H/S).subs(S, sqrtS**2)
for i in range(max_order+1):
    H_.append(((h.series(1/sqrtS, 0,i+1).removeO() - h.series(1/sqrtS, 0,i).removeO()).subs(sqrtS, sqrt(S))*S).expand())


In [5]:
display(H_[0])
display(H_[1])
display(H_[2])
display(H_[3])
display(H_[4])

-J*S*sin(theta) + S*U*cos(theta)**2/2 - S*\gamma_-*cos(theta)/2 - S*\lambda**2*cos(theta)**2/(8*\hbar\omega)

-sqrt(2)*J*sqrt(S)*cos(theta)*AnnihilateBoson(a)/2 - sqrt(2)*J*sqrt(S)*cos(theta)*CreateBoson(a)/2 - sqrt(2)*sqrt(S)*U*sin(theta)*cos(theta)*AnnihilateBoson(a)/2 - sqrt(2)*sqrt(S)*U*sin(theta)*cos(theta)*CreateBoson(a)/2 + sqrt(2)*sqrt(S)*\gamma_-*sin(theta)*AnnihilateBoson(a)/4 + sqrt(2)*sqrt(S)*\gamma_-*sin(theta)*CreateBoson(a)/4 + sqrt(2)*sqrt(S)*\lambda**2*sin(theta)*cos(theta)*AnnihilateBoson(a)/(8*\hbar\omega) + sqrt(2)*sqrt(S)*\lambda**2*sin(theta)*cos(theta)*CreateBoson(a)/(8*\hbar\omega)

J*sin(theta)*CreateBoson(a)*AnnihilateBoson(a) + U*sin(theta)**2*AnnihilateBoson(a)*CreateBoson(a)/4 + U*sin(theta)**2*AnnihilateBoson(a)**2/4 + U*sin(theta)**2*CreateBoson(a)*AnnihilateBoson(a)/4 + U*sin(theta)**2*CreateBoson(a)**2/4 - U*cos(theta)**2*CreateBoson(a)*AnnihilateBoson(a) + \gamma_-*cos(theta)*CreateBoson(a)*AnnihilateBoson(a)/2 + \hbar\omega*CreateBoson(c)*AnnihilateBoson(c) - \lambda*sin(theta)*AnnihilateBoson(a)*AnnihilateBoson(c)/4 - \lambda*sin(theta)*AnnihilateBoson(a)*CreateBoson(c)/4 - \lambda*sin(theta)*CreateBoson(a)*AnnihilateBoson(c)/4 - \lambda*sin(theta)*CreateBoson(a)*CreateBoson(c)/4 + \lambda**2*cos(theta)**2*CreateBoson(a)*AnnihilateBoson(a)/(4*\hbar\omega)

sqrt(2)*J*cos(theta)*CreateBoson(a)*AnnihilateBoson(a)**2/(8*sqrt(S)) + sqrt(2)*J*cos(theta)*CreateBoson(a)**2*AnnihilateBoson(a)/(8*sqrt(S)) + sqrt(2)*U*sin(theta)*cos(theta)*AnnihilateBoson(a)*CreateBoson(a)*AnnihilateBoson(a)/(4*sqrt(S)) + sqrt(2)*U*sin(theta)*cos(theta)*CreateBoson(a)*AnnihilateBoson(a)*CreateBoson(a)/(4*sqrt(S)) + 3*sqrt(2)*U*sin(theta)*cos(theta)*CreateBoson(a)*AnnihilateBoson(a)**2/(8*sqrt(S)) + 3*sqrt(2)*U*sin(theta)*cos(theta)*CreateBoson(a)**2*AnnihilateBoson(a)/(8*sqrt(S)) - sqrt(2)*\gamma_-*sin(theta)*CreateBoson(a)*AnnihilateBoson(a)**2/(16*sqrt(S)) - sqrt(2)*\gamma_-*sin(theta)*CreateBoson(a)**2*AnnihilateBoson(a)/(16*sqrt(S)) - sqrt(2)*\lambda*cos(theta)*CreateBoson(a)*AnnihilateBoson(a)*AnnihilateBoson(c)/(4*sqrt(S)) - sqrt(2)*\lambda*cos(theta)*CreateBoson(a)*AnnihilateBoson(a)*CreateBoson(c)/(4*sqrt(S)) - sqrt(2)*\lambda**2*sin(theta)*cos(theta)*CreateBoson(a)*AnnihilateBoson(a)**2/(32*sqrt(S)*\hbar\omega) - sqrt(2)*\lambda**2*sin(theta)*cos(theta)*Cr

-U*sin(theta)**2*AnnihilateBoson(a)*CreateBoson(a)*AnnihilateBoson(a)**2/(16*S) - U*sin(theta)**2*AnnihilateBoson(a)*CreateBoson(a)**2*AnnihilateBoson(a)/(16*S) - U*sin(theta)**2*CreateBoson(a)*AnnihilateBoson(a)**2*CreateBoson(a)/(16*S) - U*sin(theta)**2*CreateBoson(a)*AnnihilateBoson(a)**3/(16*S) - U*sin(theta)**2*CreateBoson(a)**2*AnnihilateBoson(a)*CreateBoson(a)/(16*S) - U*sin(theta)**2*CreateBoson(a)**2*AnnihilateBoson(a)**2/(8*S) - U*sin(theta)**2*CreateBoson(a)**3*AnnihilateBoson(a)/(16*S) + U*cos(theta)**2*CreateBoson(a)*AnnihilateBoson(a)*CreateBoson(a)*AnnihilateBoson(a)/(2*S) + \lambda*sin(theta)*CreateBoson(a)*AnnihilateBoson(a)**2*AnnihilateBoson(c)/(16*S) + \lambda*sin(theta)*CreateBoson(a)*AnnihilateBoson(a)**2*CreateBoson(c)/(16*S) + \lambda*sin(theta)*CreateBoson(a)**2*AnnihilateBoson(a)*AnnihilateBoson(c)/(16*S) + \lambda*sin(theta)*CreateBoson(a)**2*AnnihilateBoson(a)*CreateBoson(c)/(16*S)

In [6]:
from sympy import solve
t = symbols('t', real = True)
t_ = solve((H_[0].diff(th).subs(sin(th), 2*t/(1+t**2)).subs(cos(th), (1-t**2)/(1+t**2))*(1+t**2)**2*2*hw/S).simplify(), t)
freqs = [sqrt(J*(J*(1+t**2)**2/(4*t**2) + 2*U*t/(1+t**2))) for t in t_] 

In [ ]:
# Verifies the analytical formula for frequency reproduces the numerical result from the paper:
# Quantum Theory of the Josephson Junction between Finite Islands

from scipy.linalg import eigh_tridiagonal
import numpy as np
from matplotlib import pyplot as plt
from tqdm import tqdm
from sympy import Integer, solve

def E(k, EC, EJ, ng, N):
    D = EC * (np.arange(-N, N+1)-ng)**2
    n = np.arange(-N, N)
    O = -(EJ/(2*N)) * np.sqrt(N*(N+1) - (n*(n+1)))
    eigs =  eigh_tridiagonal(D, O, eigvals_only = True, select = 'i', select_range = (k, k), tol = 1e-14)
    return eigs[0]

EC=1
EJ=10**3
N=10**6/2
ngs = np.array([int(ng) for ng in np.linspace(-N, N, 102)[1:-1]])
numerical = np.array([E(1, EC, EJ, ng, N) - E(0, EC, EJ, ng, N) for ng in tqdm(ngs)])

t = symbols('t', real = True)
t_ = solve((H_[0].diff(th).subs(sin(th), 2*t/(1+t**2)).subs(cos(th), (1-t**2)/(1+t**2))*(1+t**2)**2*2*hw/S).simplify(), t)
freqs = [sqrt(J*(J*(1+t**2)**2/(4*t**2) + 2*U*t/(1+t**2))) for t in t_] 
EC = Integer(EC)
EJ = Integer(EJ)
N = Integer(N)
ngs = np.array([Integer(ng) for ng in ngs])
analytical = np.array([complex(freqs[3].subs({J: EJ/N, U: 2*N*EC, hw: 1, l: 0, gm: 4*ng*EC, S: N}).evalf()) for ng in tqdm(ngs)])

In [ ]:

plt.plot(ngs/N, numerical / float(sqrt(2*EJ*EC)), label = 'numerical', color = 'black')
plt.plot(ngs/N, analytical / float(sqrt(2*EJ*EC)), label = 'analytical', linestyle = '--', color = 'red')
plt.xlabel(r'$n_\mathrm{g}/N$')
plt.ylabel(r'$\hbar \omega_\mathrm{q}~[\sqrt{2E_\mathrm{J}E_\mathrm{C}}$')
plt.title(r'$2N=10^6,\quad E_\mathrm{J}/E_\mathrm{C}=10^3$')
plt.legend()
plt.show()

In [68]:
e0 = ((gm/2)*cos(th)+J*sin(th)+(U/2)*sin(th)**2 - (U-l**2/(4*hw))*cos(th)**2)
d = (U/4)*sin(th)**2
lt = (-l/4)*sin(th)
e0, d, lt = symbols('epsilon_0, Delta, lambda_t', real = True)
Ep, Em = symbols('E_+, E_-', positive = True)
Np, Nm = symbols('N_+, N_-', positive = True)
Ep_sub = sqrt((hw**2+e0**2-4*d**2)/2 + sqrt(((hw**2-e0**2+4*d**2)/2)**2 + 4*lt**2*hw*(e0-2*d)))
Em_sub = sqrt((hw**2+e0**2-4*d**2)/2 - sqrt(((hw**2-e0**2+4*d**2)/2)**2 + 4*lt**2*hw*(e0-2*d)))
Np_sub = 1/(sqrt(4*(e0-2*d)*Ep)*sqrt(1+4*hw*lt**2*(e0-2*d)/(hw**2-Ep**2)**2))
Nm_sub = 1/(sqrt(4*(e0-2*d)*Em)*sqrt(1+4*hw*lt**2*(e0-2*d)/(hw**2-Em**2)**2))
qp = B('qp')
qpd = Bd('qp')
qm = B('qm')
qmd = Bd('qm')
a_sub = Np*((Ep+e0-2*d)*qp+(-Ep+e0-2*d)*qpd) + Nm*((Em+e0-2*d)*qm+(-Em+e0-2*d)*qmd)
ad_sub = Np*((Ep+e0-2*d)*qpd+(-Ep+e0-2*d)*qp) + Nm*((Em+e0-2*d)*qmd+(-Em+e0-2*d)*qm)
c_sub = Np*((2*lt*(e0-2*d))/(Ep-hw)*qp + (2*lt*(e0-2*d))/(-Ep-hw)*qpd) + Nm*((2*lt*(e0-2*d))/(Em-hw)*qm + (2*lt*(e0-2*d))/(-Em-hw)*qmd)
cd_sub = Np*((2*lt*(e0-2*d))/(Ep-hw)*qpd + (2*lt*(e0-2*d))/(-Ep-hw)*qp) + Nm*((2*lt*(e0-2*d))/(Em-hw)*qmd + (2*lt*(e0-2*d))/(-Em-hw)*qm)
H_2 = e0*(ad*a) + d*(ad*ad+a*a)+lt*(a+ad)*(c+cd)+hw*cd*c+d
H_2 = wick(H_2.subs({a: a_sub, ad: ad_sub, c: c_sub, cd: cd_sub}).expand())

In [115]:
extract = lambda t1, t2: Add(*[x for x in H_2.args if x.has(t1) and x.has(t2)]).subs({t1:1, t2:1})
for t1, t2 in [(qp, qm), (qpd, qm), (qp, qmd), (qpd, qmd)]: 
    assert(0 == (extract(t1, t2)*(Ep+hw)*(Ep-hw)*(Em+hw)*(Em-hw)/Np/Nm).subs({Ep:Ep_sub, Em:Em_sub}).simplify())
for i in range(2):
    assert(Ep == extract(qp, qpd).subs({Np:Np_sub, lt:solve(Ep-Ep_sub, lt)[i]}).simplify())
    assert(Em == extract(qm, qmd).subs({Nm:Nm_sub, lt:solve(Em-Em_sub, lt)[i]}).simplify())
print('Verification complete.')

Verification complete.


In [ ]:
H_2 = wick(H_2)
H_2 - vev(H_2)
from sympy import collect
print(0==Add(*[x for x in H_2.args if qp in x.args and qm in x.args]).subs({qp:1, qm:1}).subs({Ep: Ep_sub, Em: Em_sub, Np: Np_sub, Nm: Nm_sub}).simplify())
print(0==Add(*[x for x in H_2.args if qpd in x.args and qmd in x.args]).subs({qpd:1, qmd:1}).subs({Ep: Ep_sub, Em: Em_sub, Np: Np_sub, Nm: Nm_sub}).simplify())
print(0==Add(*[x for x in H_2.args if qpd in x.args and qm in x.args]).subs({qpd:1, qm:1}).subs({Ep: Ep_sub, Em: Em_sub, Np: Np_sub, Nm: Nm_sub}).simplify())
print(0==Add(*[x for x in H_2.args if qp in x.args and qmd in x.args]).subs({qp:1, qmd:1}).subs({Ep: Ep_sub, Em: Em_sub, Np: Np_sub, Nm: Nm_sub}).simplify())
#display(Add(*[x for x in H_2.args if qm in x.args and qmd in x.args]).subs({qm:1, qmd:1}).simplify())#.subs({Ep: Ep_sub, Em: Em_sub, Np: Np_sub, Nm: Nm_sub}).simplify())

In [44]:
H_2 = wick(H_2)
testp=((Add(*[x for x in H_2.args if qp in x.args and qpd in x.args]).subs({qp:1, qpd:1}))).expand().subs(Np, Np_sub).subs(lt, solve(Ep-Ep_sub, lt)[0]).simplify()
display(testp)
testm=((Add(*[x for x in H_2.args if qm in x.args and qmd in x.args]).subs({qm:1, qmd:1}))).expand().subs(Nm, Nm_sub).subs(lt, solve(Em-Em_sub, lt)[0]).simplify()
display(testm)
#new_test = new_test.expand().subs(Ep, Ep_sub).expand().simplify()
#display(new_test.simplify().subs({Ep: Ep_sub, Em: Em_sub, Np: Np_sub, Nm: Nm_sub}).expand())
#for arg in test.args:
#    display(arg.factor())
#(test*(Ep+hw)**2*(Ep-hw)**2/N**2).expand()


E_+*(E_+**2 - \hbar\omega**2)/(4*Delta**2 + 2*E_+**2 - \hbar\omega**2 - epsilon_0**2)

E_-*(E_-**2 - \hbar\omega**2)/(4*Delta**2 + 2*E_-**2 - \hbar\omega**2 - epsilon_0**2)

In [20]:
solve(Ep-Ep_sub, lt**2)[0]

(-4*Delta**2*E_+**2 + 4*Delta**2*\hbar\omega**2 - E_+**4 + E_+**2*\hbar\omega**2 + E_+**2*epsilon_0**2 - \hbar\omega**2*epsilon_0**2)/(4*\hbar\omega*(2*Delta - epsilon_0))

In [ ]:
import sympy as sp
d = symbols('Delta')
e0, e1 = symbols('E_0 E_1')
eps = symbols('epsilon_0')
M = sp.Matrix([[e0-eps, -2*d, -l, -l], [2*d, e0+eps, l, l], [-l, -l, e0-hw, 0], [l, l, 0, e0+hw]])
x=  M.eigenvects()
for i in range(4): display(x[i][2][0])

In [ ]:
Es = solve(M.det(),e0)
x = M.subs(e0, Es[0]).nullspace()

In [ ]:
v = M.eigenvects()

In [ ]:
e0_val = solve(v[0][0], e0)[0]
x = sp.Matrix(v[0][2]).subs(e0, e0_val)

In [ ]:
x.simplify()

In [ ]:
x.norm().simplify()

In [ ]:
m = sp.Matrix([[e0 - eps - l**2 * (1/(e0 - hw) - 1/(e0+hw)), -2*d-l**2 * (1/(e0 - hw) - 1/(e0+hw))],[2*d + l**2 * (1/(e0 - hw) - 1/(e0+hw)), e0 + eps + l**2 * (1/(e0 - hw) - 1/(e0+hw))]])

In [ ]:
v = m.eigenvects()
display(v[0][2][0])
display(v[1][2][0])
display(v[0][0])
display(v[1][0])

In [ ]:
display(solve(v[0][0], e0)[0])
display(solve(v[0][0], e0)[1])
display(solve(v[1][0], e0)[0])
display(solve(v[1][0], e0)[1])

In [ ]:
solve(M.det(), e0)[0]

In [ ]:
v[0][2][0].norm().simplify()

In [ ]:
t1 = (e0 + eps -2*d)/(sqrt(4*(eps-2*d)*e0)*sqrt(1+4*hw*l**2*(eps-2*d)/(e0**2 - hw**2)**2)).simplify()
t2 = -(e0 - eps +2*d)/(sqrt(4*(eps-2*d)*e0)*sqrt(1+4*hw*l**2*(eps-2*d)/(e0**2 - hw**2)**2)).simplify()

In [ ]:
e0s = solve(M.det(), e0)
for i in range(len(e0s)): 
    display(e0s[i].subs(l, solve(e0s[i], l**2)[0]))

In [ ]:
e0s_positive = [e0s[1],e0s[3]]
for i in range(len(e0s_positive)): 
    display(e0s_positive[i])

In [ ]:
((e0s_positive[0]**2+2*d**2-hw**2/2-eps**2/2)**2).subs(l,0).simplify()

In [ ]:
((eps**2/2+hw**2/2-2*d**2)**2).expand()

In [ ]:
display(e0s_positive[0])
display(e0s_positive[1])

In [ ]:
sum([(x**2-hw**2)/((x**2-hw**2)**2+4*hw*l**2*(eps-2*d)) for x in e0s_positive]).simplify()

In [ ]:
sum([1/(1+4*hw*l**2*(eps-2*d)/(x**2-hw**2)**2) for x in e0s_positive]).simplify()

In [ ]:
sum([l**2*hw*(eps-2*d)/((x**2-hw**2)**2+4*hw*l**2*(eps-2*d)) for x in e0s_positive]).simplify()

In [ ]:
((e0s_positive[0]**2+2*d**2-hw**2/2-eps**2/2)**2).subs(l,0).simplify()

In [ ]:
((2*d**2+hw**2/2-eps**2/2)**2).expand()

In [ ]:
(sqrt((hw**2+eps**2-4*d**2)/2 + sqrt((hw**2-eps**2+4*d**2)**2/4 + 4*hw*l**2*(eps-2*d))) / e0s_positive[1]).simplify()
(sqrt((hw**2+eps**2-4*d**2)/2 - sqrt((hw**2-eps**2+4*d**2)**2/4 + 4*hw*l**2*(eps-2*d))) / e0s_positive[0]).simplify()

In [ ]:
e0 = 
wick(H_[2]).subs(b, )